# 必要なライブラリのインポート

In [12]:
import pickle
import numpy as np
import pandas as pd

# Notebookの中で複数の結果を表示できるようにする
from IPython import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

# 関数定義

In [13]:
# リストを入力，リストの項目で同じ項目をカウント，個数の降順にソート
def count_items(in_list):
    out_dict = {}
    for d in in_list:
        if d not in out_dict.keys():
            out_dict[d] = 1
        else:
            out_dict[d] += 1
    out_list = sorted([[x, out_dict[x]] for x in list(out_dict.keys())], key = lambda x: x[1], reverse = True)
    return out_list

# Future Net Webの日付データ(タイムスタンプデータとして入力)から，yyyymm形式の年月データに変換
def ts_ym(ts_in):
    year = str(ts_in.year)
    month = ('0' + str(ts_in.month))[-2:]
    return year + month

# リスト中の要素のNaN (数字ではないデータ)を除去（NaNが浮動小数点データであることを利用)
def rem_nan(in_list):
    out_list = []
    for i in in_list:
        if type(i) == str:
            out_list.append(i)
    return out_list

# リストの中に含まれる，ESA，鉄，HIF-PHIについて[Epo, DA, CERA, HIF, Fe]の形で用量を抽出
# 例：「ｴﾎﾟｼﾞﾝ注3000単位/0.5mL」
#  →「注」で分割，2つ目の要素を更に「単」で分割，１つ目の要素を整数に変換

def esa_fe_sum(in_list):
    esa_fe = [0, 0, 0, 0, 0]
    for in_d in in_list:
        if in_d[:6] == 'ｴﾎﾟｼﾞﾝ':
            esa_fe[0] += int(in_d.split('注')[1].split('単')[0])
        if in_d[:4] == 'ﾈｽﾌﾟ':
            esa_fe[1] += int(in_d.split('用')[1].split('μ')[0])
        if in_d[:6] == '注）ﾈｽﾌﾟ':
            esa_fe[1] += int(in_d.split('〔')[1].split('〕')[0])
        if in_d[:4] == 'ﾐﾙｾﾗ':
            esa_fe[2] += int(in_d.split('ﾞ')[1].split('μ')[0])
        if in_d.find('ﾌｪｼﾞﾝ') != -1:
            if in_d.find('静注') != -1:
                esa_fe[4] += int(in_d.split('注')[1].split('m')[0])
            else:
                esa_fe[4] += int(in_d.split('ﾝ')[1].split('m')[0])
    return esa_fe

# エクセルファイル読み込み→DataFrame

In [11]:
# IDは暗号化してあり，それぞれのエクセルファイルで共通
path = 'ここに，エクセルファイルがあるパスを，例えば，windowsなら，c:/Users/NAME/Documents/のような形で記載'

# 治療経過(Future Net Webからの出力データ)
# 各列のデータ['ID', '入外', '透析日', '透析予定時間(分)', 'ＤＷ(Kg)', '前回体重(Kg)', '前体重(Kg)',
#       '後体重(Kg)', '体重変化', '透析前最高血圧(mmHg)', '透析前最低血圧(mmHg)', '透析前平均血圧(mmHg)',
#       '透析後最高血圧(mmHg)', '透析後最低血圧(mmHg)', '透析後平均血圧(mmHg)', '再循環率', 'ダイアライザ名称',
#       '血流量(mL/min)', '抗凝固剤名称', '透析液名称', '補液名称', '消耗品4', '消耗品5', '消耗品6',
#       '消耗品7', '薬剤1', '薬剤2', '薬剤3', '薬剤4', '薬剤5', '薬剤6', '薬剤7', '薬剤8', '薬剤9',
#       '薬剤10', '薬剤11', '薬剤12', '薬剤13', '薬剤14', '薬剤15', '薬剤16', '薬剤17', '薬剤18',
#       '薬剤19', '薬剤20']
course_df = pd.read_excel(path + 'course_encrypted.xlsx')

# 検査データ(診療データ抽出データ)
# 各列のデータ['ID', '日付', 'WBC', 'RBC', 'Hgb', 'Ht', 'MCV', 'MCH', 'MCHC', 'RDW', 'Alb', 'CRP', 'TSAT', 'Ferritin']
ld_df = pd.read_excel(path + 'labo_data_encrypted.xlsx')

# それぞれのデータフレームの表示
course_df.columns
course_df.head()

ld_df.columns
ld_df.head()

# 月ごと注射データの抽出

In [10]:
# 治療経過のデータフレームをリストに変換(本来はpandas上で行うべきですが)
course_list = course_df.to_numpy().tolist()
names = course_df.columns.to_list()

# IDと年月をキーとした辞書形式で，
# 月ごとに，注射が投与された治療回数と，投与された薬剤の名称・種類・用量をリストに作成

out_dict = {}
for l in course_list:
    if str(l[0]) in out_dict.keys():
        if ts_ym(l[2]) in out_dict[str(l[0])].keys():
            out_dict[str(l[0])][ts_ym(l[2])][1] += rem_nan(l[25:45])
            out_dict[str(l[0])][ts_ym(l[2])][0] += 1
        else:
            out_dict[str(l[0])][ts_ym(l[2])] = [1, rem_nan(l[25:45])]
    else:
        out_dict[str(l[0])] = {ts_ym(l[2]): [1, rem_nan(l[25:45])]}

# 一つ目のデータを表示
k_1st = list(out_dict.keys())[0]
print(k_1st, out_dict[k_1st])

# 投与薬剤の文字列からESA, Feの投与量の総和を算出

In [9]:
# 関数esa_fe_sum()を使用して，ESA，Feの投与量の月ごとの合計を算出

out_dict_sum = {}
for k in out_dict.keys():
    for ym in out_dict[k].keys():
        data_ym = out_dict[k][ym]
        if k not in out_dict_sum.keys():
            out_dict_sum[k] = {ym : [data_ym[0], esa_fe_sum(data_ym[1])]}
        else:
            out_dict_sum[k][ym] = [data_ym[0], esa_fe_sum(data_ym[1])]
            
print(k_1st, out_dict_sum[k_1st])

# 検査データ

In [7]:
# 一人の患者の１カ月ごとのデータを一つの要素としたリストにまとめる．

ld_list_tsat = ld_df.dropna(subset = ['TSAT']).to_numpy().tolist()
ld_list_out = [[str(d[0]), ts_ym(d[1])] + d[2:]         # ID, yyyymm, 検査データ系列
               + [out_dict_sum[d[0]][ts_ym(d[1])][0]]   # 投与量総和の辞書から，ID, 年月をキーにして投薬回数を抽出→結合
               + out_dict_sum[d[0]][ts_ym(d[1])][1]     # 実際の投与量のリストを結合
                  for d in ld_list_tsat]
print(ld_list_out[:2])

# 検査データが10件以上ある患者のIDリスト
ids_selected = sorted([y[0] for y in count_items([x[0] for x in ld_list_out]) if y[1] > 10])

# 検査データが10件以上ある患者のデータを古い順にリスト化
labdata_by_pt = []
for i in ids_selected:
    temp_list = []
    for d in ld_list_out:
        if d[0] == i:
            temp_list.append(d)
    labdata_by_pt.append(sorted(temp_list, key = lambda x: x[1]))
print(labdata_by_pt[0])

In [6]:
# ESAを等価量に変換(Epo/(DA or CERA) = 250)してESA量を１データにまとめる．
#['ID', '白血球数', '赤血球数', 'ﾍﾓｸﾞﾛﾋﾞﾝ', 'ﾍﾏﾄｸﾘｯﾄ', 'MCV', 'MCH', 'MCHC', 'RDW', 
#'ｱﾙﾌﾞﾐﾝ', 'CRP定量', 'TSAT', 'ﾌｪﾘﾁﾝ', 'eqESA_dose', 'Fe_dose']

data_eq_esa = []
for dd in labdata_by_pt:
    pt_data = [[d[0]] + d[2:14] 
               + [d[15] + (d[16] + d[17])*250]  # ESA等価量
               + [d[19]] # 鉄剤総量
               for d in dd]
    data_eq_esa.append(pt_data)
print(data_eq_esa[0][0])

## 関数定義

In [19]:
# 隣り合うリストの要素間でのデータの差分計算
def diff_data(num, data_in, col, start_pos):
    ret_list = []
    start_data = data_in[start_pos][col]
    for j in range(num):
        ret_list += [data_in[start_pos + j + 1][col] - start_data]
    return ret_list

# 数値の符号を算出（マイナス→ -1，プラス → 1)
def plus_minus(data_value):
    if data_value == 0:
        return 0
    else:
        return int(data_value/abs(data_value))

# 隣り合う複数個のリストの要素をまとめる([1月のHgb，2月のHgb，3月のHgb])，NaNも除去
def extract_data(num, data_in, cols, start_pos):
    ret_list = [[] for i in range(len(cols))]
    for j in range(num):
        for i, col in enumerate(cols):
            value_append = data_in[start_pos + j][col]
            if type(value_append) == str:
                ret_list[i] += [value_append]
            elif not np.isnan(value_append):
                ret_list[i] += [value_append]
            else:
                ret_list = False
                break
        if not ret_list:
            break
    return ret_list

In [5]:
# ESA，鉄の先月との増減を最後に追加
data_with_diff = []
for pt in data_eq_esa:
    temp_list = []
    for i in range(len(pt)-1):
        temp_list += [pt[i+1] + [plus_minus(diff_data(1, pt, 13, i)[0]), plus_minus(diff_data(1, pt, 14, i)[0])]]
    data_with_diff += [temp_list]
print(len(data_with_diff))
print(data_with_diff[0][0])

In [4]:
# 連続する4か月分のデータを検査項目ごとにまとめる

col_n = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
data_to_analyze = []
for pt in data_with_diff:
    for j in range(len(pt) - 4):
        value_append = extract_data(4, pt, col_n, j)
        if value_append:
            data_to_analyze += [value_append]

print(len(data_to_analyze))
print(data_to_analyze[0])

In [3]:
# 各データを１次元につなげる
data_to_analyze_flat = []
for dd in data_to_analyze:
    temp_list = []
    for d in dd:
        temp_list += d
    data_to_analyze_flat.append(temp_list)

print(data_to_analyze_flat[0])

In [2]:
# 列名の作成
col_n_base = ['WBC', 'RBC', 'Hgb', 'Ht', 'MCV', 'MCH', 'MCHC', 'RDW', 'Alb', 'CRP', 'TSAT', 'Fer', 
          'eqESA_dose', 'Fe_dose', 'ESA_change', 'Fe_change']
col_names = []
for c in col_n_base:
    for i in range(4):
        col_names += [c + '_' + str(i + 1)]

# データフレームに変換
df = pd.DataFrame(data_to_analyze_flat, columns = col_names)
df.head()

# pickle形式で保存
with open(path + 'df_for_pycaret.pickle', 'wb') as f:
    pickle.dump(df, f)